<i><b>Public-AI</b></i>

### ✎&nbsp;&nbsp;week 4. DNN Basis

# Section 2. Keras 이해하기

### _Objective_

* Tensorflow의 Deep Learning API인 Keras를 단계 별로 살펴보며, 어떤 설계 구조를 가지고 있는지를 살펴 보도록 하겠습니다.

In [ ]:
import numpy as np
import tensorflow as tf

# [ 예제 모델 : 4층 신경망 ] 
---


<img src="https://imgur.com/U5QmvFo.png" width="500" > 

## 1. `Layer` 

딥러닝에서의 `층`은 연산의 단위입니다. `층` 단위로 연산이 전파되고, 가중치가 관리됩니다. 이러한 층의 형태는 사람이 설계해야 하는데, 위와 같이 unit 수, activation의 종류, bias의 유무가 바로 사람이 설계해야 하는 요소, Hyper-Parameter입니다.

### (1) 레이어 생성하기

In [ ]:
from tensorflow.keras.layers import Dense

hidden1_layer = Dense(5, activation='relu')
hidden2_layer = Dense(5, activation='relu')
hidden3_layer = Dense(5, activation='relu')
output_layer = Dense(1)

### (2) 레이어의 hyperparameter 가져오기 

이러한 Hyper Parameter는 `.get_config()`을 통해 확인할 수 있습니다.

In [ ]:
hidden3_layer.get_config()

### (2) `Weight` 생성하기

각 은닉층 별로 Weight는 어떤 식으로 구성되어야 할까요?

In [ ]:
print("은닉층 1: ", hidden1_layer.get_weights())
print("은닉층 2: ", hidden2_layer.get_weights())
print("은닉층 3: ", hidden3_layer.get_weights())
print("출력층  : ", output_layer.get_weights())

아예 Weight가 없다고 나옵니다. 그 이유는 무엇일까요? 바로 "입력의 갯수"가 정해져 있지 않았기 때문입니다. 각 은닉층의 Weight의 크기는 (# 입력층 내 유닛의 수, # 유닛의 수)로 결정되는데, 각 층 별 입력이 몇개의 유닛으로 이루어져있는지를 아직 지정하지 않았습니다.

#### `build` : Layer 내 Weight를 구성하기

그래서 별도로 강제로 `build`를 동작시키면, 가중치가 결정됩니다.

In [ ]:
hidden1_layer.build((None, 3))

hidden1_layer.get_weights()

입력층의 갯수가 커지면, 당연히 weight의 크기도 커집니다.

In [ ]:
hidden1_layer.build((None, 5))

hidden1_layer.get_weights()

혹은 첫번째 연산 시의 입력값 형태에 맞춰서 Weight가 초기화됩니다.

In [ ]:
hidden1_layer = Dense(5, activation='relu')
hidden1_layer.get_weights()

In [ ]:
# 입력값 예시
X = tf.constant([[0.2,0.5,0.7],
                 [0.3,0.6,0.2,],
                 [0.1,-.3,-.2]], tf.float32)

hidden1_layer(X)

In [ ]:
hidden1_layer.get_weights()

당연한 얘기이지만 가중치 내 갯수와 맞지 않은 입력값이 들어가면 아래와 같이 Error를 반환합니다.

In [ ]:
wrong_x = tf.constant([[0.2,0.5,0.7,1.2],
                 [0.3,0.6,0.2,2.3],
                 [0.1,-.3,-.2,-.5]], tf.float32)

hidden1_layer(wrong_x)

### (3) `Layer`을 통해 연산하기

`Layer`가 선언되었으면, 그이후로 Layer의 인스턴스는 하나의 Function과 같이 동작한다고 생각하면 됩니다.

In [ ]:
X1 = tf.constant([[0.2,0.5,0.7],
                 [0.3,0.6,0.2,],
                 [0.1,-.3,-.2]], tf.float32)

X2 = tf.constant([[0.3,0.6,0.2,],
                  [0.2,0.5,0.7],
                  [-.3,-.5,0.7]], tf.float32)

In [ ]:
Z1 = hidden1_layer(X1)
print(Z1)

In [ ]:
Z2 = hidden1_layer(X2)
print(Z2)

### (4) `Layer` 내 가중치를 학습시키기

가중치를 학습시키기 위해서는 순전파와 역전파를 알고 있어야 합니다. 

In [ ]:
X1 = tf.constant([[0.2,0.5,0.7],
                 [0.3,0.6,0.2,],
                 [0.1,-.3,-.2]], tf.float32)

X2 = tf.constant([[0.3,0.6,0.2,],
                  [0.2,0.5,0.7],
                  [-.3,-.5,0.7]], tf.float32)

Y_true = tf.constant([[1.2],[-0.5],[0.3]], tf.float32)

In [ ]:
# 은닉층 구성하기
hidden1_layer = Dense(5, activation='relu')
hidden2_layer = Dense(5, activation='relu')
hidden3_layer = Dense(1)

# 순전파 진행하기
with tf.GradientTape() as tape:
    Z1 = hidden1_layer(X1)
    Z2 = hidden2_layer(Z1)
    Y_pred = hidden3_layer(Z2)    
    
    loss = tf.reduce_mean((Y_true - Y_pred)**2)

# 모든 가중치 가져오기
weights = [
    *hidden1_layer.trainable_weights,
    *hidden2_layer.trainable_weights,
    *hidden3_layer.trainable_weights
]    
# 가중치의 기울기 계산하기
grads = tape.gradient(loss, weights)

# 경사하강법 적용하기
lr = 1e-4
for weight, grad in zip(weights, grads):
    weight.assign_sub(lr*grad)

## 2. `Model`

위와 같이 `Layer` 단위로 연산할 수 있게 만든 것이 `Keras`. 그래서 연산과 가중치로 이루어져 있다고 생각하면 됩니다. 하지만 딥러닝은 수십~수백개의 Layer로 이루어져 있기 때문에, 수십~수백개의 Layer를 관리하는 것이 필요합니다. 그것이 바로 `Model`입니다.

### (1) `Model`의 입력층, `Input`

`Model`은 입력과 출력만을 결정하면, 내부 로직에 의해 관련된 `Layer`들을 모두 수집하여 관리합니다. 그래서 입력만을 관리하는 특수한 녀석이 따로 존재하고 이것이 바로 `Input`입니다.

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras import backend as K

In [ ]:
K.clear_session()

inputs = Input((3,))

# 은닉층 선언
hidden1_layer = Dense(5, activation='relu', name='hidden_1')
hidden2_layer = Dense(5, activation='relu', name='hidden_2')
hidden3_layer = Dense(1, name='hidden_3')

In [ ]:
hidden1 = hidden1_layer(inputs)
hidden2 = hidden2_layer(hidden1)
hidden3 = hidden3_layer(hidden2)

### (2) 모델 선언하기

모델은 선언하는 것은 간단합니다. `Input`층부터 이어져서 출력층까지의 연결을 만들 수 있습니다.

In [ ]:
from tensorflow.keras.models import Model

model = Model(inputs, hidden1)
model.summary()

In [ ]:
model = Model(inputs, hidden2)
model.summary()

In [ ]:
model = Model(inputs, hidden3)
model.summary()

이 때 주의해야 하는 것은 `Input`층부터 시작해야 한다는 것입니다. 아니면 아래와 같이 에러를 반환합니다.

In [ ]:
Model(hidden1, hidden3)

그럼 아래와 같이 중간층부터 시작하고 싶다면 새로운 Input층을 선언해준 후 연결해주어야 합니다.

In [ ]:
hidden1_inputs = Input((5,))

# 은닉층 연결하기
hidden2 = hidden2_layer(hidden1_inputs)
hidden3 = hidden3_layer(hidden2)

model2 = Model(hidden1_inputs, hidden3)

model2.summary()

### (3) 모델 내 레이어 가져오기

`.get_layer`를 통해 간단히 가져올 수 있습니다. 

In [ ]:
layer = model.get_layer('hidden_2')

layer is hidden2_layer

In [ ]:
layer = model.get_layer('hidden_3')

layer is hidden3_layer

해당 Layer의 출력 텐서를 가져오고 싶으면, `.output`을 이용하면 됩니다.

In [ ]:
layer.output

그리고 전체적인 레이어에 대한 정보는 `model.get_config()`로 가져올 수 있습니다.

In [ ]:
model.get_config()

Tensorflow에서 모델의 구조는 내부적으로 `Graph`의 형태로 관리됩니다. 하지만 Keras API에서는 config json 포맷으로 관리됩니다. 그래서 Tensorflow를 저장할 때에는 모델의 연산을 정의하는 Graph와 모델 내 가중치를 의미하는 Checkpoint를 따로 저장했다면, Keras에서는 모델의 연산을 정의하는 Config 와 모델 내 가중치를 의미하는 weights를 따로 저장합니다.

### (3) 모델에서 관리하고 있는 가중치 가져오기

현재 모델에서의 가중치 중에서 학습해야할 가중치는 `trainable_variables`에 있습니다.

In [ ]:
model.trainable = True

model.trainable_variables

In [ ]:
model.trainable = False

model.trainable_variables

대신 학습대상이 아닌 가중치들은 `non_trianable_variables`에 들어갑니다.

In [ ]:
model.non_trainable_variables

우리는 종종 Pretrain Model 등을 이용할 때, 특정 층만 학습에서 제외해야 할 때가 있습니다. 그런 경우 우리는 Model 내 Layer 별로 trainable를 지정해줄 수 있습니다.

In [ ]:
model.trainable = True
model.get_layer('hidden_1').trainable = False
model.get_layer('hidden_2').trainable = True
model.get_layer('hidden_3').trainable = False

model.summary()

`.get_weights()`로 가져오는 것은 모델 자체 내 현재 가중치의 값을 넘파이로 가져올 때 사용합니다.

In [ ]:
model.get_weights()

### (4) 모델 전체 순전파하기

In [ ]:
X1 = tf.constant([[0.2,0.5,0.7],
                 [0.3,0.6,0.2,],
                 [0.1,-.3,-.2]], tf.float32)

X2 = tf.constant([[0.3,0.6,0.2,],
                  [0.2,0.5,0.7],
                  [-.3,-.5,0.7]], tf.float32)

Y_true = tf.constant([[1.2],[-0.5],[0.3]], tf.float32)

In [ ]:
Y_pred = model(X1)
Y_pred

`predict`는 추론환경에서 이용하기 위해 만들어진 특수한 연산입니다. 바로 Numpy로 반환하게 됩니다.

In [ ]:
model.predict(X1)

### (5) 모델 역전파하기

In [ ]:
with tf.GradientTape() as tape:
    # model.predict(X1)하면 동작하지 않음
    Y_pred = model(X1)
    Loss = tf.reduce_mean((Y_pred-Y_true)**2)

# 역전파로 기울기 구하기
grads = tape.gradient(Loss, model.trainable_weights)

# 경사하강법 적용하기
lr = 1e-4
for weight, grad in zip(model.trainable_weights, grads):
    weight.assign_sub(lr*grad)

이 과정에서 어떤 손실함수를 이용할지, 어떤 경사하강법을 이용할지, 어떤 Metricd으로 평가할지를 결정하게 됩니다. 이 과정을 케라스에서는 `compile`이라는 함수를 통해 일괄적으로 지정할 수 있습니다.

In [ ]:
model.compile(optimizer='sgd',loss='mean_squared_error')

그리고 데이터를 통해 모델 내 가중치를 업데이트 하는 것은 바로 `fit`으로 진행하게 됩니다.

In [ ]:
model.fit(X, Y_pred)

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2020/03/26

---